### Import necessary libraries

In [33]:
import pandas as pd
import numpy as np
import psycopg2

### Read in CSV data

In [34]:
# Read in both data files and display the first five rows
df_movies = pd.read_csv("movies.csv")
df_imdb = pd.read_csv("imdb_top_1000.csv")
display(df_movies.head())
display(df_imdb.head())

,MOVIES,YEAR,GENRE,STARS,VOTES,Gross
0,Blood Red Sky,-2021,"\nAction, Horror, Thriller",\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure","\n \n Stars:\nChris Wood, \nSara...","17,870",NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller","\n \n Stars:\nAndrew Lincoln, \n...","885,805",NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy","\n \n Stars:\nJustin Roiland, \n...","414,849",NaN
4,Army of Thieves,-2021,"\nAction, Crime, Horror",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0


### Text Cleaning: Removing unnecessary characters from specific columns 

In [35]:
# Use replace to remove new line character
df_movies = df_movies.replace(r'\n',' ', regex=True)

# Replaced pipe character with semi colon
df_movies["STARS"] = df_movies["STARS"].replace(r'\|',';', regex=True)

# Replaced double comma with a single comma
df_movies = df_movies.replace(',,',',', regex=True)

# Show first five rows of new file
df_movies.head()

,MOVIES,YEAR,GENRE,STARS,VOTES,Gross
0,Blood Red Sky,-2021,"Action, Horror, Thriller",Director: Peter Thorwarth ; Stars: P...,"21,062",NaN
1,Masters of the Universe: Revelation,(2021– ),"Animation, Action, Adventure","Stars: Chris Wood, Sarah Mi...","17,870",NaN
2,The Walking Dead,(2010–2022),"Drama, Horror, Thriller","Stars: Andrew Lincoln, Norm...","885,805",NaN
3,Rick and Morty,(2013– ),"Animation, Adventure, Comedy","Stars: Justin Roiland, Chri...","414,849",NaN
4,Army of Thieves,-2021,"Action, Crime, Horror",Director: Matthias Schweighöfer ; St...,NaN,NaN


### Extract Directors and Stars into there own columns

In [36]:
# Function to extract Director and Stars
def extract_director_and_stars(row):

    # Split the row into "Director" and "Stars" parts
    parts = row.split(';')
    
    # Extract the Director and Stars based on identifiable patterns
    director = [part.replace('Director:', '').strip() for part in parts if 'Director' in part]
    stars = [part.replace('Stars:', '').strip() for part in parts if 'Stars' in part]
    
    # Join the extracted parts (just in case there's more than one part, but typically there shouldn't be)
    director = ', '.join(director)
    stars = ', '.join(stars)
    
    # Return the cleaned-up director and stars
    return pd.Series([director, stars])

# Apply the extraction function to the STARS column and create new columns
df_movies[['Director', 'Stars']] = df_movies['STARS'].apply(extract_director_and_stars)
# Remove the original STARS column
df_movies = df_movies.drop(columns=['STARS'])

# Clean up extraneous semicolons and spaces in Director and Stars columns
df_movies['Director'] = df_movies['Director'].str.replace(r'\s*,\s*', ', ')
df_movies['Stars'] = df_movies['Stars'].str.replace(r'\s*,\s*', ', ')  # Ensure there's exactly one space after each comma
df_movies.head()

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars
0,Blood Red Sky,-2021,"Action, Horror, Thriller","21,062",NaN,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander ..."
1,Masters of the Universe: Revelation,(2021– ),"Animation, Action, Adventure","17,870",NaN,,"Chris Wood, Sarah Michelle Gellar, Lena Head..."
2,The Walking Dead,(2010–2022),"Drama, Horror, Thriller","885,805",NaN,,"Andrew Lincoln, Norman Reedus, Melissa McBri..."
3,Rick and Morty,(2013– ),"Animation, Adventure, Comedy","414,849",NaN,,"Justin Roiland, Chris Parnell, Spencer Gramm..."
4,Army of Thieves,-2021,"Action, Crime, Horror",NaN,NaN,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ru..."


### Cleaning up Year column

In [37]:
# Use regular expression to extract only the first four letter numeric string and convert to object
df_movies['YEAR'] = df_movies['YEAR'].str.extract(r'(\d{4})').astype(object)

# Fill NaN values with "Unknown"
df_movies["YEAR"] = df_movies["YEAR"].fillna("Unknown")
df_movies

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars
0,Blood Red Sky,2021,"Action, Horror, Thriller","21,062",NaN,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander ..."
1,Masters of the Universe: Revelation,2021,"Animation, Action, Adventure","17,870",NaN,,"Chris Wood, Sarah Michelle Gellar, Lena Head..."
2,The Walking Dead,2010,"Drama, Horror, Thriller","885,805",NaN,,"Andrew Lincoln, Norman Reedus, Melissa McBri..."
3,Rick and Morty,2013,"Animation, Adventure, Comedy","414,849",NaN,,"Justin Roiland, Chris Parnell, Spencer Gramm..."
4,Army of Thieves,2021,"Action, Crime, Horror",NaN,NaN,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ru..."
...,...,...,...,...,...,...,...
6810,The Imperfects,2021,"Adventure, Drama, Fantasy",NaN,NaN,,"Morgan Taylor Campbell, Chris Cope, Iñaki Go..."
6811,AlRawabi School for Girls,2021,Drama,NaN,NaN,Tima Shomali,"Salsabiela A., Joanna Arida, Yara Mustafa, ..."
6812,Totenfrau,2022,"Drama, Thriller",NaN,NaN,Nicolai Rohde,"Felix Klare, Romina Küper, Anna Maria Mühe, ..."
6813,Arcane,2021,"Animation, Action, Adventure",NaN,NaN,,


In [38]:
# Function to convert values in millions (M) to whole numbers
def convert_millions(value):
    if isinstance(value, str):
        if 'M' in value:
            # Replace 'M', convert to float and multiply by 1,000,000
            return int(float(value.replace('M', '')[1:]) * 1000000)
        else:
            return value  # Return the original value if not in millions

# Apply the conversion function to the 'Amount' column
df_movies['Gross'] = df_movies['Gross'].apply(convert_millions)
# Format the currency appropriately
df_movies["Gross"] = df_movies["Gross"].map('${0:,.0f}'.format)
df_movies["Gross"] = df_movies["Gross"].replace("$nan", np.nan)

# Replace any nan values in votes column with 0 and remove commas from numbers
df_movies["VOTES"] = df_movies["VOTES"].replace(np.nan, "0")
df_movies["VOTES"] = df_movies["VOTES"].str.replace(",", "")

In [39]:
# Remove any rows with $0 gross amount
df_movies = df_movies[df_movies["Gross"] != "$0"]
df_movies = df_movies.drop_duplicates(["MOVIES", "YEAR"])

# Regular expression to match numeric-like patterns: percentages, dates, times
pattern = r'\d{1,4}%|\d{1,2}/\d{1,2}/\d{2,4}|\d{1,2}:\d{2}(?:\s?[APMapm]{2})?|\d{4}'
df_movies = df_movies[~df_movies['MOVIES'].str.contains(pattern, regex=True)]

### Export the cleaned filed to a new CSV

In [40]:
df_movies.to_csv("cleaned_movies.csv", index=False)
df_movies

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars
0,Blood Red Sky,2021,"Action, Horror, Thriller",21062,NaN,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander ..."
1,Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",17870,NaN,,"Chris Wood, Sarah Michelle Gellar, Lena Head..."
2,The Walking Dead,2010,"Drama, Horror, Thriller",885805,NaN,,"Andrew Lincoln, Norman Reedus, Melissa McBri..."
3,Rick and Morty,2013,"Animation, Adventure, Comedy",414849,NaN,,"Justin Roiland, Chris Parnell, Spencer Gramm..."
4,Army of Thieves,2021,"Action, Crime, Horror",0,NaN,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ru..."
...,...,...,...,...,...,...,...
6810,The Imperfects,2021,"Adventure, Drama, Fantasy",0,NaN,,"Morgan Taylor Campbell, Chris Cope, Iñaki Go..."
6811,AlRawabi School for Girls,2021,Drama,0,NaN,Tima Shomali,"Salsabiela A., Joanna Arida, Yara Mustafa, ..."
6812,Totenfrau,2022,"Drama, Thriller",0,NaN,Nicolai Rohde,"Felix Klare, Romina Küper, Anna Maria Mühe, ..."
6813,Arcane,2021,"Animation, Action, Adventure",0,NaN,,


In [41]:
# Import cleaned movies file
cleaned_movies = pd.read_csv("cleaned_movies.csv")

# Strip the string values in the movies column of extra spaces and make them all lowercase
cleaned_movies['MOVIES'] = cleaned_movies['MOVIES'].apply(lambda x: x.strip().lower())
cleaned_movies

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars
0,blood red sky,2021,"Action, Horror, Thriller",21062,NaN,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander ..."
1,masters of the universe: revelation,2021,"Animation, Action, Adventure",17870,NaN,NaN,"Chris Wood, Sarah Michelle Gellar, Lena Head..."
2,the walking dead,2010,"Drama, Horror, Thriller",885805,NaN,NaN,"Andrew Lincoln, Norman Reedus, Melissa McBri..."
3,rick and morty,2013,"Animation, Adventure, Comedy",414849,NaN,NaN,"Justin Roiland, Chris Parnell, Spencer Gramm..."
4,army of thieves,2021,"Action, Crime, Horror",0,NaN,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ru..."
...,...,...,...,...,...,...,...
6735,the imperfects,2021,"Adventure, Drama, Fantasy",0,NaN,NaN,"Morgan Taylor Campbell, Chris Cope, Iñaki Go..."
6736,alrawabi school for girls,2021,Drama,0,NaN,Tima Shomali,"Salsabiela A., Joanna Arida, Yara Mustafa, ..."
6737,totenfrau,2022,"Drama, Thriller",0,NaN,Nicolai Rohde,"Felix Klare, Romina Küper, Anna Maria Mühe, ..."
6738,arcane,2021,"Animation, Action, Adventure",0,NaN,NaN,NaN


In [42]:
# Strip the string values in the movies column of extra spaces and make them all lowercase
df_imdb['Series_Title'] = df_imdb['Series_Title'].apply(lambda x: x.strip().lower())
df_imdb

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,the shawshank redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,the godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,the dark knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,the godfather: part ii,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 angry men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0
...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,breakfast at tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,from here to eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0


### Merge the IMDB and Movies datasets

In [43]:
# Merge both datasets on both the movie name and year it was released columns to ensure unique identification
merged_cleaned_movies = pd.merge(cleaned_movies, df_imdb, left_on=["MOVIES", "YEAR"], right_on=["Series_Title", "Released_Year"])

# Drop the extra columns from the second data frame that are common between both datasets
merged_cleaned_movies = merged_cleaned_movies.drop(["Released_Year", "Series_Title", "Genre"], axis=1)

# Export the cleaned dataset to a CSV file
merged_cleaned_movies.to_csv("merged_cleaned_movies.csv", index=False)
merged_cleaned_movies.head()

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars,Poster_Link,Certificate,Runtime,IMDB_Rating,Overview,Meta_score
0,jurassic park,1993,"Action, Adventure, Sci-Fi",897444,"$402,450,000",Steven Spielberg,"Sam Neill, Laura Dern, Jeff Goldblum, Richa...",https://m.media-amazon.com/images/M/MV5BMjM2MD...,UA,127 min,8.1,A pragmatic paleontologist visiting an almost ...,68.0
1,the lord of the rings: the fellowship of the ring,2001,"Action, Adventure, Drama",1713028,"$315,540,000",Peter Jackson,"Elijah Wood, Ian McKellen, Orlando Bloom, S...",https://m.media-amazon.com/images/M/MV5BN2EyZj...,U,178 min,8.8,A meek Hobbit from the Shire and eight compani...,92.0
2,jaws,1975,"Adventure, Thriller",558731,"$260,000,000",Steven Spielberg,"Roy Scheider, Robert Shaw, Richard Dreyfuss,...",https://m.media-amazon.com/images/M/MV5BMmVmOD...,A,124 min,8.0,When a killer shark unleashes chaos on a beach...,87.0
3,the departed,2006,"Crime, Drama, Thriller",1227522,"$132,380,000",Martin Scorsese,"Leonardo DiCaprio, Matt Damon, Jack Nicholso...",https://m.media-amazon.com/images/M/MV5BMTI1MT...,A,151 min,8.5,An undercover cop and a mole in the police att...,85.0
4,gone girl,2014,"Drama, Mystery, Thriller",895888,"$167,770,000",David Fincher,"Ben Affleck, Rosamund Pike, Neil Patrick Har...",https://m.media-amazon.com/images/M/MV5BMTk0MD...,A,149 min,8.1,With his wife's disappearance having become th...,79.0


### Create database schema and table and add CSV data to table

In [44]:
# Connect to the JHU postgre database
DB_NAME = "jhu"
DB_USER = "jhu"
DB_PASS = "jhu123"
DB_HOST = "localhost"
DB_PORT = "5432"

try:
    conn = psycopg2.connect(
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        host=DB_HOST,
        port=DB_PORT
    )
    print("Database connected successfully")
except:
    print("Database not connected successfully")

# Generate a cursor for executing SQL commands on the database
cur = conn.cursor()

# Execute SQL to create a new schema and table for the merged, cleaned file
cur.execute("""

    CREATE SCHEMA IF NOT EXISTS imdb;
    
    DROP TABLE imdb.Movies;
    
    CREATE TABLE IF NOT EXISTS imdb.Movies (
            Movies TEXT NOT NULL,
            Year INT NOT NULL,
            Genre TEXT,
            Votes INT,
            Gross TEXT,
            Director TEXT,
            Stars TEXT,
            PosterLink TEXT,
            Certificate VARCHAR(5),
            Runtime VARCHAR(7),
            IMDB_Rating DECIMAL(2, 1),
            Overview TEXT,
            Meta_Score DECIMAL(4, 1)
    );

""")

# Commit the cursor execution SQL to the database
conn.commit()
print("Table Created successfully")

# Send the data from the CSV file to the database table 
for index, row in merged_cleaned_movies.iterrows():
    cur.execute("""
        INSERT INTO imdb.Movies (Movies, Year, Genre, Votes, Gross, Director, Stars, PosterLink, Certificate, Runtime, IMDB_Rating, Overview, Meta_Score) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, 
    (row['MOVIES'], row['YEAR'], row['GENRE'], row['VOTES'], row['Gross'], row['Director'], row['Stars'], row['Poster_Link'], row['Certificate'], row['Runtime'], row['IMDB_Rating'], row['Overview'], row['Meta_score']))

# Commit the data insertion to the database
conn.commit()
print("Data Loaded successfully")

Database connected successfully
Table Created successfully
Data Loaded successfully


### Use SQL query to ensure data was added correctly

In [45]:
# Create a cursor and query the first five rows from the table
cur = conn.cursor()
cur.execute("SELECT * FROM imdb.movies LIMIT 5")
rows = cur.fetchall()

# Create basic dataframe with row data and same column names as merged, cleaned data
pd.DataFrame(data=rows, columns=merged_cleaned_movies.columns)

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars,Poster_Link,Certificate,Runtime,IMDB_Rating,Overview,Meta_score
0,jurassic park,1993,"Action, Adventure, Sci-Fi",897444,"$402,450,000",Steven Spielberg,"Sam Neill, Laura Dern, Jeff Goldblum, Richa...",https://m.media-amazon.com/images/M/MV5BMjM2MD...,UA,127 min,8.1,A pragmatic paleontologist visiting an almost ...,68.0
1,the lord of the rings: the fellowship of the ring,2001,"Action, Adventure, Drama",1713028,"$315,540,000",Peter Jackson,"Elijah Wood, Ian McKellen, Orlando Bloom, S...",https://m.media-amazon.com/images/M/MV5BN2EyZj...,U,178 min,8.8,A meek Hobbit from the Shire and eight compani...,92.0
2,jaws,1975,"Adventure, Thriller",558731,"$260,000,000",Steven Spielberg,"Roy Scheider, Robert Shaw, Richard Dreyfuss,...",https://m.media-amazon.com/images/M/MV5BMmVmOD...,A,124 min,8.0,When a killer shark unleashes chaos on a beach...,87.0
3,the departed,2006,"Crime, Drama, Thriller",1227522,"$132,380,000",Martin Scorsese,"Leonardo DiCaprio, Matt Damon, Jack Nicholso...",https://m.media-amazon.com/images/M/MV5BMTI1MT...,A,151 min,8.5,An undercover cop and a mole in the police att...,85.0
4,gone girl,2014,"Drama, Mystery, Thriller",895888,"$167,770,000",David Fincher,"Ben Affleck, Rosamund Pike, Neil Patrick Har...",https://m.media-amazon.com/images/M/MV5BMTk0MD...,A,149 min,8.1,With his wife's disappearance having become th...,79.0


In [46]:
# compare database data to the actual CSV file
merged_cleaned_movies.head()

,MOVIES,YEAR,GENRE,VOTES,Gross,Director,Stars,Poster_Link,Certificate,Runtime,IMDB_Rating,Overview,Meta_score
0,jurassic park,1993,"Action, Adventure, Sci-Fi",897444,"$402,450,000",Steven Spielberg,"Sam Neill, Laura Dern, Jeff Goldblum, Richa...",https://m.media-amazon.com/images/M/MV5BMjM2MD...,UA,127 min,8.1,A pragmatic paleontologist visiting an almost ...,68.0
1,the lord of the rings: the fellowship of the ring,2001,"Action, Adventure, Drama",1713028,"$315,540,000",Peter Jackson,"Elijah Wood, Ian McKellen, Orlando Bloom, S...",https://m.media-amazon.com/images/M/MV5BN2EyZj...,U,178 min,8.8,A meek Hobbit from the Shire and eight compani...,92.0
2,jaws,1975,"Adventure, Thriller",558731,"$260,000,000",Steven Spielberg,"Roy Scheider, Robert Shaw, Richard Dreyfuss,...",https://m.media-amazon.com/images/M/MV5BMmVmOD...,A,124 min,8.0,When a killer shark unleashes chaos on a beach...,87.0
3,the departed,2006,"Crime, Drama, Thriller",1227522,"$132,380,000",Martin Scorsese,"Leonardo DiCaprio, Matt Damon, Jack Nicholso...",https://m.media-amazon.com/images/M/MV5BMTI1MT...,A,151 min,8.5,An undercover cop and a mole in the police att...,85.0
4,gone girl,2014,"Drama, Mystery, Thriller",895888,"$167,770,000",David Fincher,"Ben Affleck, Rosamund Pike, Neil Patrick Har...",https://m.media-amazon.com/images/M/MV5BMTk0MD...,A,149 min,8.1,With his wife's disappearance having become th...,79.0
